In [ ]:
import tensorflow as tf

IMG_SIZE = 224
NUM_FEATURES = 7 * 7 * 1280
NUM_CLASSES = 62


class TransferLearningModel(tf.Module):
    def __init__(self):
        super().__init__()
        self.num_features = NUM_FEATURES
        self.num_classes = NUM_CLASSES
        
        self.base_model = tf.keras.applications.MobileNetV2(
            input_shape=(IMG_SIZE, IMG_SIZE, 3),
            include_top=False,
            alpha=1.0,
            weights='imagenet'
        )
        self.base_model.trainable = False
        
        self.model = tf.keras.Sequential([
            self.base_model,
            tf.keras.layers.Conv2D(32, 3, activation='relu', name='MY_CONVV'),
            tf.keras.layers.Dropout(0.2, name='MY_DROPP'),
            tf.keras.layers.GlobalAveragePooling2D(name='MY_GAGAAPP'),
            tf.keras.layers.Dense(62, activation='softmax', name='MY_DENSEE')
        ])
        self.loss_fn = tf.keras.losses.CategoricalCrossentropy()
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.0005)

    def load_trained_weights(self, weights_path):
        sample_input = tf.zeros((1, 224, 224, 3))
        _ = self.model(sample_input)
        
        self.model.load_weights(weights_path)
        
        print(f"Number of trainable variables = {len(self.model.trainable_variables)}")
        for var in self.model.trainable_variables:
            print(f"Variable: {var.name}")


    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, IMG_SIZE, IMG_SIZE, 3], dtype=tf.float32)
    ])
    def load(self, feature):
        x = tf.keras.applications.mobilenet_v2.preprocess_input(
        tf.multiply(feature, 255))
        bottleneck = tf.reshape(
        self.base_model(x, training=False), (-1, self.num_features))
        return {"bottleneck": bottleneck}




    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, NUM_FEATURES], dtype=tf.float32),
        tf.TensorSpec(shape=[None, NUM_CLASSES], dtype=tf.float32)
    ])
    def train(self, bottleneck, y):
        with tf.GradientTape() as tape:
            x = tf.reshape(bottleneck, [-1, 7, 7, 1280])
            
            x = self.model.get_layer('MY_CONVV')(x, training=True)
            x = self.model.get_layer('MY_DROPP')(x, training=True)  
            x = self.model.get_layer('MY_GAGAAPP')(x, training=True)
            predictions = self.model.get_layer('MY_DENSEE')(x, training=True)
            
            loss = self.loss_fn(y, predictions)

        
        trainable_vars = []
        for layer in self.model.layers[1:]:
            trainable_vars.extend(layer.trainable_variables)
            
        gradients = tape.gradient(loss, trainable_vars)
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        return {"loss": loss}




    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, IMG_SIZE, IMG_SIZE, 3], dtype=tf.float32)
    ])
    def infer(self, feature):
        x = tf.keras.applications.mobilenet_v2.preprocess_input(
        tf.multiply(feature, 255))
    
        bottleneck = self.base_model(x, training=False) 
        
        x = self.model.get_layer('MY_CONVV')(bottleneck, training=False)
        x = self.model.get_layer('MY_DROPP')(x, training=False)
        x = self.model.get_layer('MY_GAGAAPP')(x, training=False)
        predictions = self.model.get_layer('MY_DENSEE')(x, training=False)
        
        return {"output": predictions}

    @tf.function(input_signature=[
        tf.TensorSpec(shape=[None, NUM_FEATURES], dtype=tf.float32)
    ])
    def infer_with_bottleneck(self, bottleneck):
        x = tf.reshape(bottleneck, [-1, 7, 7, 1280])
        
        x = self.model.get_layer('MY_CONVV')(x, training=False)
        x = self.model.get_layer('MY_DROPP')(x, training=False)  
        x = self.model.get_layer('MY_GAGAAPP')(x, training=False)
        predictions = self.model.get_layer('MY_DENSEE')(x, training=False)
        
        return {"output": predictions}


    @tf.function(input_signature=[])
    def save(self):
        return {"status": tf.constant("saved")}

    @tf.function(input_signature=[])
    def restore(self):
        return {"status": tf.constant("restored")}

tflite_model = TransferLearningModel()

tflite_model.load_trained_weights('baseline_model.weights.h5')

dummy_image = tf.zeros((1, 224, 224, 3))
dummy_bottleneck = tf.zeros((1, NUM_FEATURES))
dummy_labels = tf.zeros((1, 62))

_ = tflite_model.load(dummy_image)
_ = tflite_model.train(dummy_bottleneck, dummy_labels)
_ = tflite_model.infer(dummy_image)

tf.saved_model.save(tflite_model, 'trained_handwriting_savedmodel', signatures={
    'load': tflite_model.load,
    'train': tflite_model.train,
    'infer': tflite_model.infer,
    'infer_with_bottleneck': tflite_model.infer_with_bottleneck,
    'save': tflite_model.save,
    'restore': tflite_model.restore,
})

converter = tf.lite.TFLiteConverter.from_saved_model('trained_handwriting_savedmodel')
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS
]
converter.experimental_enable_resource_variables = True
converter.allow_custom_ops = True

tflite_model_data = converter.convert()

with open('handwriting_model.tflite', 'wb') as f:
    f.write(tflite_model_data)

print(f"Model size: {len(tflite_model_data) / (1024 * 1024):.1f} MB")
